In [18]:
# Combine Data Across All Sessions (for a given participant)

In [19]:
import pandas as pd
import numpy as np
from pandas import *
from scipy import stats
import os
import re

In [20]:
# get column names & types from sample file

In [21]:
file = '../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Data/All Synchronized/P3_annos/annos_s20_2017-11-08/other_data/Combined_s20_2017-11-08.csv'

init_data = pd.read_csv(file)

/anaconda3/envs/caispp/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,6,7,8,9,16,17,18,19,20,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [22]:
of_op_cols = []

for i in init_data.columns:
    if 'of' in i or 'op' in i:
        of_op_cols.append(i)

save_columns = ['engagement', 'timestamp', 'participant', 'session_num', 'session_date', 'activity', 'difficulty', 'ros_ROBOT_STATE', 'ros_PARTICIPANT_STATE', 'transcript_spk_0', 'transcript_spk_1', 'transcript_spk_2']

new_columns = ['mistake_made', 'game_incorrect', 'game_correct', 'game_start']

master_cols = save_columns + new_columns + of_op_cols

In [23]:
master = pd.DataFrame(columns = master_cols)

### Precondition Calculations
- game start
    - new continuous instance of "game2_act1_inst0" in ros_ROBOT_STATE
- game end (moving on)
    - "Movingon" in ros_PARTICIPANT_STATE
- game end (completed successfully)
    - "GameCompleted" in ros_PARTICIPANT_STATE
- mistake made
    - new continuous instance of "Madeamistake in ros_PARTICIPANT_STATE"

In [24]:
rootdir = '../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Data/All Synchronized/P9_annos'

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if 'Combined' in file: #one of the files to process     
            print(file)
                        
            path_to_file = subdir + '/' + file
            data = pd.read_csv(path_to_file)
            
            new_columns = ['mistake_made', 'game_incorrect', 'game_correct', 'game_start', 'session_date', 'session_num']

            for c in new_columns:
                if c != 'session_date':
                    data[c] = 0.0
                else:
                    data[c] = ''
            
            for i,r in data.iterrows():
                ros_robot = r['ros_ROBOT_STATE']
                ros_participant = r['ros_PARTICIPANT_STATE']

                # new continuous instance in ros_ROBOT_STATE 
                if (isinstance(ros_robot, str)) and ((i==0) or (data.at[i-1, 'ros_ROBOT_STATE'] != ros_robot)): 
                    # new game_start
                    if ('game' in ros_robot) and ('act' in ros_robot) and  ('inst' in ros_robot):
                        data.at[i, 'game_start'] = 1

                # new continuous instance in ros_PARTICIPANT_STATE
                if (isinstance(ros_participant, str)) and ((i==0) or (data.at[i-1, 'ros_PARTICIPANT_STATE'] != ros_participant)):
                    # new game correct 
                    if ('GameCompleted' in ros_participant):
                        data.at[i, 'game_correct'] = 1

                    # new game incorrect
                    if ('Movingon' in ros_participant):
                        data.at[i, 'game_incorrect'] = 1

                    # mistake made
                    if ('Madeamistake' in ros_participant):
                        data.at[i, 'mistake_made'] = 1
                        
                # extract session num and date
                session = r['session']
                nan_indices = []
                
                if ('s18-19') in file:
                    data.at[i, 'session_num'] = 18
                    data.at[i, 'session_date'] = '2018-05-14'
                elif (isinstance(session, str)):
                    date_regex = '201\d-\d\d-\d\d'
                    date = re.findall(date_regex, session)[0]

                    num_regex = 's\d*_'
                    num = re.findall(num_regex, session)[0]
                    num = num[1:-1]


                    data.at[i, 'session_num'] = num
                    data.at[i, 'session_date'] = date
                else:
                    nan_indices.append(i)
            
            data = data.drop(data.index[nan_indices])
            data = data.drop(columns=['session'])
            
            data['timestamp'] = data['Timestamp']
            data = data.drop(columns=['Timestamp'])
            
            print(data.shape)
            print(master.shape)
            
            master = pd.concat([master,data], sort=True)

Combined_s23_2018-06-27.csv


/anaconda3/envs/caispp/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(81110, 1553)
(0, 1553)
Combined_s16_2018-06-25.csv


/anaconda3/envs/caispp/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(18392, 1553)
(81110, 1553)
Combined_s6_2018-06-12.csv
(9363, 1553)
(99502, 1553)
Combined_s9_2018-06-14.csv
(21974, 1553)
(108865, 1553)
Combined_s8_2018-06-14.csv
(18918, 1553)
(130839, 1553)
Combined_s7_2018-06-12.csv
(20681, 1553)
(149757, 1553)
Combined_s4_2018-06-11.csv


/anaconda3/envs/caispp/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(18003, 1553)
(170438, 1553)
Combined_s3_2018-06-07.csv
(58622, 1553)
(188441, 1553)
Combined_s1_2018-06-07.csv
(30074, 1553)
(247063, 1553)


In [25]:
path_data = '../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/p9_data.csv'

master.to_csv(path_data, index=False)